In [ ]:
# with tf.control_dependencies([x_op]):

In [1]:


import tensorflow as tf
import numpy as np




In [2]:
x = tf.Variable(0.0)
x_plus_1 = tf.assign_add(x, 1)

with tf.control_dependencies([x_plus_1]):
    y = x
init = tf.initialize_all_variables()

with tf.Session() as session:
    init.run()
    for i in xrange(5):
        print(y.eval())

Instructions for updating:
Use `tf.global_variables_initializer` instead.
0.0
0.0
0.0
0.0
0.0


In [4]:
x = tf.Variable(0.0)
x_plus_1 = tf.assign_add(x, 1)

with tf.control_dependencies([x_plus_1]):
    y = tf.identity(x)
init = tf.initialize_all_variables()

with tf.Session() as session:
    init.run()
    for i in xrange(5):
        print(y.eval())

Instructions for updating:
Use `tf.global_variables_initializer` instead.
1.0
2.0
3.0
4.0
5.0


In [3]:
x = tf.Variable(0.)
x_op = tf.assign(x, 1.)

with tf.control_dependencies([x_op]):
    true_fun  = lambda: tf.assign_add(x, 3.)
    false_fun = lambda: tf.constant([])
    pred = tf.constant(True)
    cocontrol_flow_ops.cond(pred, true_fun, false_fun)

NameError: name 'cocontrol_flow_ops' is not defined

In [ ]:


with tf.control_dependencies([loss_averages_op]):
  total_loss = tf.identity(total_loss)


In [ ]:

down vote
	

I came across another use case that is not completely covered by the other answers.

def conv_layer(input_tensor, kernel_shape, output_dim, layer_name, decay=None, act=tf.nn.relu):
    """Reusable code for making a simple convolutional layer.
    """
    # Adding a name scope ensures logical grouping of the layers in the graph.
    with tf.name_scope(layer_name):
        # This Variable will hold the state of the weights for the layer
        with tf.name_scope('weights'):
            weights = weight_variable(kernel_shape, decay)
            variable_summaries(weights, layer_name + '/weights')
        with tf.name_scope('biases'):
            biases = bias_variable([output_dim])
            variable_summaries(biases, layer_name + '/biases')
        with tf.name_scope('convolution'):
            preactivate = tf.nn.conv2d(input_tensor, weights, strides=[1, 1, 1, 1], padding='SAME')
            biased = tf.nn.bias_add(preactivate, biases)
            tf.histogram_summary(layer_name + '/pre_activations', biased)
        activations = act(biased, 'activation')
        tf.histogram_summary(layer_name + '/activations', activations)
        return activations


In [ ]:
http://stackoverflow.com/questions/41780655/difference-between-tf-group-and-tf-control-dependencies

In [ ]:
c=tf.group(a, b) #produces the same graph below that using cotrol_dependencies

with tf.control_dependencies([a, b]):
    c = tf.no_op() 


###### 1
down vote
favorite
2
	

Is it possible to create a control dependency between two ops after they've both been created? I realize that with tf.control_dependencies it's possible to make one op wait on another before executing, but the dependent op has to be created within the tf.control_dependencies context. I'd like to construct the two ops independently first and then add the dependency.


###### what is Context Manager ?

###### 위 코드에서 A는 with 블럭이 시작되는 시점에 A.enter()를 호출하고, 블럭이 끝나는 시점에 A.exit()를 호출한다. 만약 블럭 내에서 예외가 발생하여 블럭의 실행이 중지되고 빠져나오는 시점에도 A.exit()가 호출되는 것이 보장된다.

예를 들어 파일을 열어서 어떤 내용을 처리하고 파일을 닫는 동작을 수행할 때, 컨텍스트 매니저가 파일을 열고, 닫는 동작을 정의한다면, with 구문에서는 이를 컨텍스트 매니저(이 경우에는 파일 디스크립터)에게 이러한 동작을 위임하고 실제 데이터를 처리하는 코드만 작성할 수 있

In [ ]:
with g.control_dependencies([a, b]):
  # Ops constructed here run after `a` and `b`.
  with g.control_dependencies(None):
    # Ops constructed here run normally, not waiting for either `a` or `b`.
    with g.control_dependencies([c, d]):
      # Ops constructed here run after `c` and `d`, also not waiting
      # for either `a` or `b`.


In [ ]:
# WRONG
def my_func(pred, tensor):
  t = tf.matmul(tensor, tensor)
  with tf.control_dependencies([pred]):
    # The matmul op is created outside the context, so no control
    # dependency will be added.
    return t

# RIGHT
def my_func(pred, tensor):
  with tf.control_dependencies([pred]):
    # The matmul op is created in the context, so a control dependency
    # will be added.
    return tf.matmul(tensor, tensor)
